In [13]:
from pyscf import gto, dft
import numpy as np
import os
import qml

import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
from alchemy_tools_pyscf import calculate_alchpot
from parse_pyscf import read_input

In [14]:
os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "4" # export NUMEXPR_NUM_THREADS=6

In [15]:
# read input file
run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/alchpot_lambda1/dsgdb9nsd_000293/'#sys.argv[1]
os.chdir(run_dir)
input_parameters = read_input(os.path.join(run_dir, 'input_parameters'))
inputfile = input_parameters['structure_file']
basis =  'def2tzvp'#input_parameters['basis'] 
com = qml.Compound(xyz=inputfile)
# create mol object and set up calculation
mol = gto.Mole()
for ch, coords_atom in zip(com.nuclear_charges, com.coordinates):
    mol.atom.append([ch, coords_atom])
mol.basis = basis
mol.verbose = 1
mol.build()
includeonly = np.arange(len(mol.atom_coords())) # necessary for alchpot function

In [16]:
# run SCF-calculation
mf = dft.RKS(mol)
mf.xc = 'lda,vwn'
mf.kernel()

-306.45539831479743

In [17]:
# calculate the alchemical potentials
dm1_ao = mf.make_rdm1()
alchpots = calculate_alchpot(dm1_ao, includeonly, mol)

In [18]:
ap2 = np.load('/home/misa/projects/Atomic-Energies/data/vacuum_reference/alchpot_lambda1/dsgdb9nsd_000293/alchpots.npy')

In [19]:
alchpots-ap2

array([ 3.55271368e-15,  1.42108547e-14, -1.77635684e-14,  7.10542736e-15,
       -5.32907052e-14,  0.00000000e+00,  8.88178420e-15,  1.77635684e-15,
       -7.10542736e-15,  1.59872116e-14, -1.95399252e-14,  5.32907052e-15,
       -7.10542736e-15, -3.55271368e-15, -3.55271368e-15, -1.24344979e-14])